# O Problema da Mochila (Knapsack Problem)

O Problema da Mochila é um problema clássico de otimização combinatória. A ideia é selecionar itens para colocar em uma mochila com capacidade limitada, de forma a maximizar o valor total dos itens escolhidos sem ultrapassar o limite de peso.

## Descrição do Problema da Mochila

- Suponha que temos uma mochila com uma capacidade de peso máxima $W$.
- Existem $n$ itens disponíveis, onde cada item $i$ possui um peso $w_i$ e um valor $v_i$.
- Nosso objetivo é maximizar o valor total dos itens escolhidos, respeitando a restrição de peso da mochila.

In [ ]:
import random

def geraPopulacaoInicial(tamanhoPopulacao, quantidadeItens):

  # Gera uma população inicial de indivíduos, onde cada indivíduo é um vetor binário
  # representando a presença (1) ou ausência (0) de um item na mochila.

  populacao = []

  for i in range(tamanhoPopulacao):

    populacao.append([random.randint(0, 1) for j in range(quantidadeItens)]) # Aleatoriamente seleciona 0 ou 1 para cada item

  return populacao

In [ ]:
from itertools import combinations

def calculaSolucaoOtima(pesosValores, capacidade):

  # Inicializa a variável para armazenar o melhor valor encontrado
  melhor_valor = 0

  # Variável que pode armazenar a combinação de itens que resulta no melhor valor
  melhor_combinacao = None

  # 'numItens' recebe o número total de itens disponíveis
  numItens = len(pesosValores)

  # Itera sobre todas as combinações possíveis de itens, com 'r' variando de 1 até o número total de itens
  for r in range(1, numItens + 1):

    # 'combinations(range(numItens), r)' gera todas as combinações possíveis de 'r' itens a partir do total de itens disponíveis
    for combinacao in combinations(range(numItens), r):

      # Calcula o peso total da combinação atual
      peso_total = sum(pesosValores[i][0] for i in combinacao)

      # Calcula o valor total da combinação atual
      valor_total = sum(pesosValores[i][1] for i in combinacao)

      # Verifica se o peso total não excede a capacidade da mochila
      if peso_total <= capacidade:

        # Se o valor total da combinação atual for maior do que o melhor valor encontrado até agora
        if valor_total > melhor_valor:

          # Atualiza o melhor valor e a melhor combinação
          melhor_valor = valor_total

          melhor_combinacao = combinacao

  # Retorna o melhor valor encontrado
  return melhor_valor

In [ ]:
def fitness(populacao, pesosValores, capacidade):

  # Calcula o valor de fitness para cada indivíduo da população.

  fits = []

  for individuo in populacao:

    # Calcula o valor total e o peso total dos itens selecionados
    peso_total = sum(peso for (peso, valor), gene in zip(pesosValores, individuo) if gene == 1)

    valor_total = sum(valor for (peso, valor), gene in zip(pesosValores, individuo) if gene == 1)

    if peso_total <= capacidade:

      fits.append(valor_total) # Adiciona o valor ao fitness se o peso for válido

    else:

      fits.append(0) # Penalidade por exceder a capacidade da mochila

  return fits

In [ ]:
def roletaViciada(populacao, fits):

  # Seleciona um indivíduo da população com base no fitness relativo, simulando uma roleta viciada.

  totalFitness = sum(fits)

  if totalFitness == 0:  # Caso todos os indivíduos sejam inválidos

    return random.choice(populacao)

  # Calcula as probabilidades relativas de cada indivíduo
  probabilidades = [fit / totalFitness for fit in fits]

  acumulado = 0

  intervalos = []

  # Cria intervalos cumulativos para simular a roleta
  for prob in probabilidades:

    acumulado += prob

    intervalos.append(acumulado)

  # Gira a roleta
  roleta = random.random()

  for i, intervalo in enumerate(intervalos):

    if roleta <= intervalo:

      return populacao[i]

In [ ]:
def selecaoNatural(populacao, fits, numeroCasais):

  # Seleciona indivíduos para reprodução com base no fitness.

  selecionados = []

  for i in range(2 * numeroCasais):  # Precisamos de dois indivíduos por casal

      selecionados.append(roletaViciada(populacao, fits))

  return selecionados

In [ ]:
def crossOver(genoma1, genoma2):

  # Realiza crossover entre dois indivíduos para gerar dois filhos.

  ponto_corte = random.randint(1, len(genoma1) - 1)  # Define um ponto de corte

  # Cria dois filhos combinando partes dos genomas dos pais
  filho1 = genoma1[:ponto_corte] + genoma2[ponto_corte:]

  filho2 = genoma2[:ponto_corte] + genoma1[ponto_corte:]

  return filho1, filho2

In [ ]:
def mutacao(individuo, taxaMutacao):

  # Aplica uma mutação em um indivíduo com uma determinada probabilidade.

  for i in range(len(individuo)):

    if random.random() < taxaMutacao:  # Decide se o gene será mutado

      individuo[i] = 1 - individuo[i]  # Alterna entre 0 e 1

  return individuo

In [ ]:
def algoritmoGenetico(pesosValores, capacidade, tamanhoPopulacao, numGeracoes, taxaMutacao, numCasais):

  # Executa o algoritmo genético para resolver o problema da mochila.

  numGenes = len(pesosValores)

  populacao = geraPopulacaoInicial(tamanhoPopulacao, numGenes)

  solucao_otima = calculaSolucaoOtima(pesosValores, capacidade)

  for geracao in range(numGeracoes):

    # Calcula o fitness da população
    fits = fitness(populacao, pesosValores, capacidade)

    # Verifica se algum indivíduo já encontrou a solução ótima
    if max(fits) == solucao_otima:

      print(f"Solução ótima encontrada na geração {geracao + 1}")

      break

    # Seleção natural
    reprodutores = selecaoNatural(populacao, fits, numCasais)

    # Crossover e geração da nova população
    novaPopulacao = []

    for i in range(0, len(reprodutores), 2):

      filho1, filho2 = crossOver(reprodutores[i], reprodutores[i + 1])

      novaPopulacao.extend([filho1, filho2])

    # Aplicação de mutação
    novaPopulacao = [mutacao(individuo, taxaMutacao) for individuo in novaPopulacao]

    # Atualiza a população
    populacao = novaPopulacao

  # Após todas as gerações, retorna o melhor indivíduo encontrado
  fits = fitness(populacao, pesosValores, capacidade)

  melhor_individuo = populacao[fits.index(max(fits))]

  return melhor_individuo, max(fits)

In [ ]:
pesosValores = [[4, 30], [8, 10], [8, 30], [25, 75], [2, 10], [50, 100], [6, 300], [12, 50], [100, 400], [8, 300]]

capacidade_mochila = 100

tamanhoPopulacao = 10

numGeracoes = 20

taxaMutacao = 0.2

numCasais = 5

resultado, valor = algoritmoGenetico(pesosValores, capacidade_mochila, tamanhoPopulacao, numGeracoes, taxaMutacao, numCasais)

print("Melhor indivíduo:", resultado)

print("Valor:", valor)

Solução ótima encontrada na geração 18
Melhor indivíduo: [1, 1, 1, 0, 1, 1, 1, 1, 0, 1]
Valor: 830


In [ ]:
pesosValores = [[5, 30], [7, 10], [8, 30], [25, 75], [3, 10], [50, 100], [6, 300], [12, 50]]

capacidade_mochila = 75

tamanhoPopulacao = 8

numGeracoes = 50

taxaMutacao = 0.3

numCasais = 4

resultado, valor = algoritmoGenetico(pesosValores, capacidade_mochila, tamanhoPopulacao, numGeracoes, taxaMutacao, numCasais)

print("Melhor indivíduo:", resultado)

print("Valor:", valor)

Solução ótima encontrada na geração 20
Melhor indivíduo: [1, 1, 1, 1, 1, 0, 1, 1]
Valor: 505
